In [1]:
# importing numpy and pandas
import numpy as np
import pandas as pd

In [2]:
# first working on user id data

user_data = open("user.dat","r").read()

In [3]:
# the file contain non ACSII and symboles that we dont need we are only keeping the data with numbers and char

a = ""
for i in user_data:
    if i.isalnum():
        a += i
    else:
        a += " "
a = a.split()

# variable a contain a list of name,id,name,id,name,id so we need 2 list name and id 
name = [a[i] for i in range(len(a)) if i%2==0]
user_id = [int(a[i]) for i in range(len(a)) if i%2==1]

# now converting this 2 list into one dictionary with id as key and name as value
user = dict(zip(user_id,name))

# converting that dictionary to data frame
user_df = pd.DataFrame(user.values(),index=user.keys())
user_df.reset_index(inplace=True)
user_df.rename(columns={0:"Name","index":"id"},inplace=True)

# our work has done we get the data of users in this variable
# user_df  

In [4]:
# this is the main file which has the attendance data and reading it in csv

df = pd.read_csv("attendance.dat",header=None)

In [5]:
# because the data is in one column spliting it and keeping only the req. two columns so make these function that extract the data in first two columns

def get_id(text):
    return text.split("\t")[0]
def get_time(text):
    return text.split("\t")[1]

In [6]:
# creating two columns ID and punch_Time that hold the data when the employee punch in and out

df["id"]= df[0].map(get_id)
df["punch_time"] = df[0].map(get_time)
df.drop(columns=[0],inplace=True)   # deleting the raw data column and keeping only two column
df['id']= df["id"].astype(int)      # conerting id column to int

In [7]:
# merge the data of id and punch time data into one data frame

data = pd.merge(df,user_df,how="inner",on="id")

In [8]:
# extracting date and time 

data['datetime'] = pd.to_datetime(df['punch_time'])
data['date'] = pd.to_datetime(df['punch_time']).dt.date
data['in_out_time'] = pd.to_datetime(df['punch_time']).dt.time
data["am_pm"] = data['datetime'].dt.strftime("%p")
data.drop(columns="punch_time",inplace=True)     # no use of this column now

In [9]:
# we have only one column of IN and OUT time of employee , so making two columns to seperate them

ampm = data.pivot_table(index=data.index,columns='am_pm',values='datetime',aggfunc="min")
ampm.reset_index(inplace=True)
ampm.rename(columns={"index":"row_id"},inplace=True)
# ampm

In [10]:
# merge both table into main data file

data = data.reset_index().rename(columns={"index":"row_id"})
data = pd.merge(data,ampm,how='inner',on ='row_id')

In [11]:
# importing timesheet of employee IN and OUT time

timesheet = pd.read_excel("timesheet.xlsx",sheet_name="time")
ts = timesheet[['Name','max late','max early']]

In [12]:
# merge time data to main dataframe

data = pd.merge(data,ts,how="inner",on ='Name')

In [13]:
data['max late'] = data['max late'].dt.time
data['max early'] = data['max early'].dt.time
data['AM'] = data['AM'].dt.time
data['PM'] = data['PM'].dt.time

In [14]:
# filling null values in AM column with max late and max early
data['AM'] = data['AM'].fillna(data['max late'])
data['PM'] = data['PM'].fillna(data['max early'])

In [15]:
data['late_arival'] = data['AM']>data['max late']
data['early leave'] = data['PM']<data['max early']

In [16]:
late_data = data.groupby("Name").agg({"late_arival":"sum","early leave":"sum"})

In [17]:
# 'ta' stand for total attendance a temprary variable 
ta = pd.DataFrame(data.groupby(["Name",'date'])['date'].count()).rename({"date":"count"},axis=1).reset_index()

In [18]:
total_present = pd.DataFrame(ta.groupby("Name")['date'].count())

In [19]:
final = pd.merge(total_present,late_data,on='Name',how='inner').reset_index()

In [20]:
final.rename(columns={"date":"total_present_day"},inplace=True)

In [21]:
final['net_present_day']=final['total_present_day']-(final['late_arival']*0.5 + final['early leave']*0.5)

In [22]:
leave_data = pd.read_excel("leave_data.xlsx")

In [23]:
leave_data = leave_data[['Name','Number of Days','Leave Start Date','Leave End Date']]

In [24]:
leave_data['half_name'] = leave_data['Name'].map(lambda x : x.split()[0])

In [25]:
# total leave taken by employees
total_leave = pd.DataFrame(leave_data.groupby('half_name')['Number of Days'].sum()).reset_index()

In [26]:
# exporting total leave data to excel
total_leave.to_csv("Total_leave.csv")

In [27]:
final = pd.merge(final,total_leave,left_on='Name',right_on="half_name",how="left")

In [28]:
final.drop(columns="half_name",inplace=True)
final.rename({"Number of Days":"Leave"},axis=1,inplace=True)
final['Leave']= final['Leave'].fillna(0)
final.to_csv("FinalReport.csv")

In [29]:
final

,Name,total_present_day,late_arival,early leave,net_present_day,Leave
0,Abhishek,24,0,0,24.0,0.0
1,Aditi,3,0,1,2.5,2.0
2,Ajay,8,3,0,6.5,0.0
3,Arvind,25,0,0,25.0,0.0
4,Bantu,23,3,1,21.0,0.0
5,Dev,9,8,2,4.0,0.0
6,Devesh,21,0,1,20.5,5.0
7,Diksha,24,0,1,23.5,0.5
8,Kuldeep,24,0,0,24.0,0.0
9,Purnima,22,0,1,21.5,3.0
